# Calculation of Earth's Gravitational Energy Change

We'll calculate the change in gravitational energy of the Earth due to a small change in its polar radius. The gravitational energy of an ellipsoidal Earth is given by the Landau-Lifshitz formula:

$$ U = -\frac{3 G M^2}{5 \sqrt{a^2 - c^2}} \arccos\left(\frac{c}{a}\right) $$

where:
- \( G \) is the gravitational constant,
- \( M \) is the mass of the Earth,
- \( a \) is the equatorial radius,
- \( c \) is the polar radius.

We'll use `mpmath` for high-precision calculations. Let's define the constants and functions first.


In [171]:
import mpmath as mp

# Set precision for mpmath
mp.dps = 20

# Constants
G_val = mp.mpf('6.67384e-11')  # Gravitational constant in m^3 kg^-1 s^-2
M_val = mp.mpf('5.9722e24')    # Mass of the Earth in kg

# Initial radii
a_init_val = mp.mpf('6378136.600') # Initial equatorial radius in meters
c_init_val = mp.mpf('6356751.858') # Initial polar radius in meters

# Function definitions
def ecvatorial_radii_calc(delta_c_val, nu=mp.mpf('0.3')):
    # Calculate the change in equatorial radius
    relative_change_c = mp.fdiv(delta_c_val, c_init_val)
    relative_change_a = mp.fmul(mp.fneg(nu), relative_change_c)
    delta_a_val = mp.fmul(relative_change_a, a_init_val)
    return delta_a_val

def gravitational_energy(a, c):
    # Calculate gravitational energy using the Landau-Lifshitz formula
    acos_arg = mp.fdiv(c, a)
    acos_val = mp.acos(acos_arg)
    sqrt_val = mp.sqrt(mp.fsub(mp.fmul(a, a), mp.fmul(c, c)))
    denominator = mp.fmul(5, sqrt_val)
    U = mp.fmul(mp.fdiv(mp.fmul(mp.fmul(-3, G_val), mp.fmul(M_val, M_val)), denominator), acos_val)
    return U

def delta_energy(delta_c_val, nu=mp.mpf('0.3')):
    # Calculate the change in gravitational energy
    delta_a_val = ecvatorial_radii_calc(delta_c_val, nu)
    U_init = gravitational_energy(a_init_val, c_init_val)
    U_new = gravitational_energy(mp.fadd(a_init_val, delta_a_val), mp.fadd(c_init_val, delta_c_val))
    delta_U = mp.fsub(U_new, U_init)
    return delta_U

# Change in polar radius
delta_c_val = mp.mpf('0.000146447144')

# Calculate the change in gravitational energy
dU = delta_energy(delta_c_val)

## Results

Now, let's evaluate the change in gravitational energy for the given change in the polar radius:


In [172]:
dU_float = float(dU)
dU_float

6.846000095761034e+20

In [174]:
import pandas as pd
import numpy as np

# Загрузка данных из файла CSV
file_path = 'prem_data.csv'
data = pd.read_csv(file_path)

# Гравитационная постоянная
G = 6.67384 * 10**-11  # м^3 кг^-1 с^-2

# Эксцентриситет Земли
e = 3.2845479 * 10**-3

# Функция для расчета массы эллипсоида
def calculate_mass(radius, density):
    return (4/3) * np.pi * density * (radius * 1000)**3  # Преобразование радиуса в метры

# Функция для расчета гравитационной энергии эллипсоида
def calculate_gravitational_energy(mass, radius):
    if radius > 1:  # Установка минимального порога для радиуса в 1 метр
        return -3/5 * G * mass**2 / (radius * 1000) * (1 - e**2 / 3)
    else:
        return 0  # Возвращаем 0 для очень маленьких или нулевых радиусов

# Расчет масс и гравитационных энергий для каждого слоя
masses = []
energies = []
for i in range(len(data) - 1):
    outer_layer = data.iloc[i]
    inner_layer = data.iloc[i + 1]

    outer_mass = calculate_mass(outer_layer['radius'], outer_layer['density'])
    inner_mass = calculate_mass(inner_layer['radius'], inner_layer['density'])
    layer_mass = outer_mass - inner_mass

    layer_energy = calculate_gravitational_energy(layer_mass, outer_layer['radius'])

    masses.append(layer_mass)
    energies.append(layer_energy)

# Суммирование всех энергий для получения общей гравитационной энергии Земли
total_gravitational_energy = sum(energies)

print(f"Общая гравитационная энергия Земли: {total_gravitational_energy} Дж")


Общая гравитационная энергия Земли без учета сжатия: -9.584217894957784e+27 Дж
